# FE 추가 탐색 노트북 (`FE_add.ipynb`)

이 노트북은 **파이프라인/모델 설계 전에, 성능 향상 가능성이 있는 FE를 마지막으로 더 찾아보기 위한 전용 실험장**입니다.

- 입력 데이터: `../data/raw_data.csv`
- 기반 FE: `FE_validation.ipynb`에서 성능이 좋았던 **핵심 FE 세트(Set D)**를 그대로 사용  
  - `engagement_score`, `songs_per_minute`, `skip_intensity`, `skip_rate_cap` 등
- 목표:
  1. 몇 가지 **세그먼트/ratio/비선형 FE 후보**를 추가로 만들어 보고,
  2. RandomForest + threshold 튜닝 기준으로 **각 후보를 1개씩 추가했을 때 F1/AUC가 얼마나 좋아지는지** 비교

> 이 노트북에서 **유의미한 개선이 없으면, 현 데이터 기준 FE 탐색은 여기서 종료**하고 파이프라인/모델 설계로 넘어가는 것을 전제로 합니다.


In [1]:
# 1. 데이터 로드 및 기본 설정

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

pd.set_option("display.max_columns", None)

DATA_PATH = "../data/raw_data.csv"

df = pd.read_csv(DATA_PATH)
print("df shape =", df.shape)
print(df.columns)


df shape = (8000, 12)
Index(['user_id', 'gender', 'age', 'country', 'subscription_type',
       'listening_time', 'songs_played_per_day', 'skip_rate', 'device_type',
       'ads_listened_per_week', 'offline_listening', 'is_churned'],
      dtype='object')


In [3]:
# 2. 기반 FE 생성 함수 (FE_validation의 Set D와 동일 구조)

NUM_COLS = [
    "age",
    "listening_time",
    "songs_played_per_day",
    "skip_rate",
    "ads_listened_per_week",
    "offline_listening",
]


def make_base_fe_df() -> pd.DataFrame:
    """결측치 median 처리 + 설계 FE 4개 + (songs_per_minute, skip_intensity)까지 생성한 df 반환
    - FE 4개: engagement_score, listening_time_bin, skip_rate_cap, ads_pressure
    - 추가 FE: songs_per_minute, skip_intensity
    """
    df_tmp = df.copy()

    # 1) 수치형 결측치 median 처리
    for c in NUM_COLS:
        if c in df_tmp.columns and df_tmp[c].isnull().any():
            df_tmp[c] = df_tmp[c].fillna(df_tmp[c].median())

    # 2) FE 4개 생성
    # 2-1) engagement_score
    if {"listening_time", "songs_played_per_day"}.issubset(df_tmp.columns):
        df_tmp["engagement_score"] = (
            df_tmp["listening_time"] * df_tmp["songs_played_per_day"]
        )

    # 2-2) listening_time_bin
    if "listening_time" in df_tmp.columns:
        try:
            df_tmp["listening_time_bin"] = pd.qcut(
                df_tmp["listening_time"], 3, labels=["low", "mid", "high"]
            )
        except Exception:
            bins = [0, 60, 180, df_tmp["listening_time"].max()]
            df_tmp["listening_time_bin"] = pd.cut(
                df_tmp["listening_time"],
                bins=bins,
                labels=["low", "mid", "high"],
                include_lowest=True,
            )

    # 2-3) skip_rate_cap
    if "skip_rate" in df_tmp.columns:
        df_tmp["skip_rate_cap"] = df_tmp["skip_rate"].clip(lower=0, upper=1.5)

    # 2-4) ads_pressure
    if {"ads_listened_per_week", "listening_time"}.issubset(df_tmp.columns):
        lt_nonzero = df_tmp["listening_time"].replace(0, np.nan)
        df_tmp["ads_pressure"] = df_tmp["ads_listened_per_week"] / lt_nonzero

    # 3) 추가 FE: songs_per_minute, skip_intensity
    lt_safe = df_tmp["listening_time"].replace(0, np.nan)
    df_tmp["songs_per_minute"] = (
        df_tmp["songs_played_per_day"] / lt_safe
    ).fillna(0.0)
    df_tmp["skip_intensity"] = df_tmp["skip_rate"] * df_tmp["songs_played_per_day"]

    return df_tmp


df_base = make_base_fe_df()
print("df_base columns:")
print(df_base.columns)


df_base columns:
Index(['user_id', 'gender', 'age', 'country', 'subscription_type',
       'listening_time', 'songs_played_per_day', 'skip_rate', 'device_type',
       'ads_listened_per_week', 'offline_listening', 'is_churned',
       'engagement_score', 'listening_time_bin', 'skip_rate_cap',
       'ads_pressure', 'songs_per_minute', 'skip_intensity'],
      dtype='object')


In [4]:
# 3. 새 후보 FE 생성 함수 (세그먼트 플래그 / ratio / 비선형)

CANDIDATE_FE_COLS = [
    "heavy_user_flag",
    "mobile_free_flag",
    "high_ads_low_time_flag",
    "ads_per_song",
    "sqrt_listening_time",
]


def add_candidate_fe(df_in: pd.DataFrame) -> pd.DataFrame:
    df_tmp = df_in.copy()

    # 3-1) heavy_user_flag: listening_time, songs_played_per_day 둘 다 상위 25%
    if {"listening_time", "songs_played_per_day"}.issubset(df_tmp.columns):
        lt_q75 = df_tmp["listening_time"].quantile(0.75)
        sp_q75 = df_tmp["songs_played_per_day"].quantile(0.75)
        df_tmp["heavy_user_flag"] = (
            (df_tmp["listening_time"] >= lt_q75)
            & (df_tmp["songs_played_per_day"] >= sp_q75)
        ).astype(int)

    # 3-2) mobile_free_flag: Mobile + Free 조합
    if {"device_type", "subscription_type"}.issubset(df_tmp.columns):
        df_tmp["mobile_free_flag"] = (
            (df_tmp["device_type"] == "Mobile")
            & (df_tmp["subscription_type"] == "Free")
        ).astype(int)

    # 3-3) ads_per_song: 광고 수 / 곡 수 (0 나누기 방지)
    if {"ads_listened_per_week", "songs_played_per_day"}.issubset(df_tmp.columns):
        df_tmp["ads_per_song"] = df_tmp["ads_listened_per_week"] / (
            df_tmp["songs_played_per_day"] + 1e-3
        )

    # 3-4) high_ads_low_time_flag: ads_pressure 상위 25% & listening_time_bin == 'low'
    if {"ads_pressure", "listening_time_bin"}.issubset(df_tmp.columns):
        ap_q75 = df_tmp["ads_pressure"].quantile(0.75)
        df_tmp["high_ads_low_time_flag"] = (
            (df_tmp["ads_pressure"] >= ap_q75)
            & (df_tmp["listening_time_bin"] == "low")
        ).astype(int)

    # 3-5) sqrt_listening_time: 비선형 변환 (주로 선형 모델용, 여기서는 참고)
    if "listening_time" in df_tmp.columns:
        df_tmp["sqrt_listening_time"] = np.sqrt(df_tmp["listening_time"].clip(lower=0))

    return df_tmp


df_all = add_candidate_fe(df_base)
print("추가 후 컬럼 수:", len(df_all.columns))


추가 후 컬럼 수: 23


In [5]:
# 4. RF + threshold 튜닝으로 성능 평가 함수 정의

CORE_FE_COLS = [
    "engagement_score",
    "songs_per_minute",
    "skip_intensity",
    "skip_rate_cap",
]


def evaluate_rf_best_f1(X, y, thresholds=None):
    if thresholds is None:
        thresholds = np.linspace(0.1, 0.9, 17)

    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    rf = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_split=5,
        class_weight="balanced",
        n_jobs=-1,
        random_state=42,
    )
    rf.fit(X_train, y_train)

    y_proba = rf.predict_proba(X_valid)[:, 1]

    best_f1 = 0.0
    best_th = 0.5
    for th in thresholds:
        y_pred_th = (y_proba >= th).astype(int)
        f1 = f1_score(y_valid, y_pred_th)
        if f1 > best_f1:
            best_f1 = f1
            best_th = th

    auc = roc_auc_score(y_valid, y_proba)
    return best_f1, best_th, auc


In [6]:
# 5. baseline(Set D) vs 후보 FE 1개씩 추가(Set H_x) 성능 비교

# y 정의
if "is_churned" not in df_all.columns:
    raise ValueError("'is_churned' 컬럼이 없습니다.")

y = df_all["is_churned"]

# Set D와 동일한 baseline feature 세트
base_num = NUM_COLS.copy()
core_feats = CORE_FE_COLS.copy()

X_base = df_all[base_num + core_feats]

base_f1, base_th, base_auc = evaluate_rf_best_f1(X_base, y)
print("[Baseline Set D] F1 = {:.4f} @ th={:.2f}, AUC = {:.4f}".format(base_f1, base_th, base_auc))

print("\n=== 후보 FE 1개씩 추가 (Set H_x) ===")
for cand in CANDIDATE_FE_COLS:
    if cand not in df_all.columns:
        continue
    X_cand = df_all[base_num + core_feats + [cand]]
    f1_c, th_c, auc_c = evaluate_rf_best_f1(X_cand, y)
    print(
        f"[+ {cand:22s}] F1 = {f1_c:.4f} (Δ {f1_c - base_f1:+.4f}) @ th={th_c:.2f}, AUC = {auc_c:.44f} (Δ {auc_c - base_auc:+.4f})"
    )


[Baseline Set D] F1 = 0.4127 @ th=0.10, AUC = 0.5289

=== 후보 FE 1개씩 추가 (Set H_x) ===
[+ heavy_user_flag       ] F1 = 0.4126 (Δ -0.0001) @ th=0.10, AUC = 0.53578993246490858037844873251742683351039886 (Δ +0.0069)
[+ mobile_free_flag      ] F1 = 0.4121 (Δ -0.0006) @ th=0.10, AUC = 0.53788360176291838410378431944991461932659149 (Δ +0.0090)
[+ high_ads_low_time_flag] F1 = 0.4079 (Δ -0.0048) @ th=0.10, AUC = 0.53286938599278210748622086612158454954624176 (Δ +0.0040)
[+ ads_per_song          ] F1 = 0.4126 (Δ -0.0001) @ th=0.15, AUC = 0.53788767504949042130846237341756932437419891 (Δ +0.0090)
[+ sqrt_listening_time   ] F1 = 0.4113 (Δ -0.0014) @ th=0.10, AUC = 0.53208935161424353310621881973929703235626221 (Δ +0.0032)


---

## 6. 추가 후보 FE v2 (극단치 플래그 + ratio의 ratio + 교호작용)

첫 실험에서 유의미한 개선이 없었으므로, **마지막 시도**로 조금 더 "극단적인" 패턴을 잡는 FE들을 추가합니다.

- **극단 행동 플래그**: 특정 조건 조합에서 극단적인 유저를 태깅
- **중첩 ratio**: 비율의 비율로 더 복잡한 관계 표현
- **교호작용**: 두 변수의 곱으로 명시적인 interaction 생성


In [7]:
# 추가 후보 FE v2 생성

CANDIDATE_FE_V2_COLS = [
    "zero_offline_high_skip",
    "very_low_engagement",
    "ads_heavy_free",
    "skip_to_engagement",
    "ads_intensity_per_minute",
    "engagement_x_skip",
    "songs_per_min_x_ads_pressure",
]


def add_candidate_fe_v2(df_in: pd.DataFrame) -> pd.DataFrame:
    df_tmp = df_in.copy()

    # === 극단 행동 플래그 ===
    
    # 1) zero_offline_high_skip: 오프라인 안 쓰면서 스킵 많이
    if {"offline_listening", "skip_rate"}.issubset(df_tmp.columns):
        skip_q75 = df_tmp["skip_rate"].quantile(0.75)
        df_tmp["zero_offline_high_skip"] = (
            (df_tmp["offline_listening"] == 0) & (df_tmp["skip_rate"] >= skip_q75)
        ).astype(int)

    # 2) very_low_engagement: engagement_score 하위 10%
    if "engagement_score" in df_tmp.columns:
        eng_q10 = df_tmp["engagement_score"].quantile(0.10)
        df_tmp["very_low_engagement"] = (
            df_tmp["engagement_score"] <= eng_q10
        ).astype(int)

    # 3) ads_heavy_free: Free 유저 중 광고 상위 25%
    if {"subscription_type", "ads_listened_per_week"}.issubset(df_tmp.columns):
        ads_q75 = df_tmp["ads_listened_per_week"].quantile(0.75)
        df_tmp["ads_heavy_free"] = (
            (df_tmp["subscription_type"] == "Free")
            & (df_tmp["ads_listened_per_week"] >= ads_q75)
        ).astype(int)

    # === 중첩 ratio (ratio의 ratio) ===
    
    # 4) skip_to_engagement: skip_intensity / (engagement_score + 1)
    if {"skip_intensity", "engagement_score"}.issubset(df_tmp.columns):
        df_tmp["skip_to_engagement"] = df_tmp["skip_intensity"] / (
            df_tmp["engagement_score"] + 1.0
        )

    # 5) ads_intensity_per_minute: ads_listened_per_week / (listening_time + 1)
    if {"ads_listened_per_week", "listening_time"}.issubset(df_tmp.columns):
        df_tmp["ads_intensity_per_minute"] = df_tmp["ads_listened_per_week"] / (
            df_tmp["listening_time"] + 1.0
        )

    # === 교호작용 (interaction) ===
    
    # 6) engagement_x_skip: engagement_score * skip_rate_cap
    if {"engagement_score", "skip_rate_cap"}.issubset(df_tmp.columns):
        df_tmp["engagement_x_skip"] = (
            df_tmp["engagement_score"] * df_tmp["skip_rate_cap"]
        )

    # 7) songs_per_min_x_ads_pressure: songs_per_minute * ads_pressure
    if {"songs_per_minute", "ads_pressure"}.issubset(df_tmp.columns):
        df_tmp["songs_per_min_x_ads_pressure"] = (
            df_tmp["songs_per_minute"] * df_tmp["ads_pressure"]
        )

    return df_tmp


df_all_v2 = add_candidate_fe_v2(df_all)
print("v2 후보 추가 후 컬럼 수:", len(df_all_v2.columns))


v2 후보 추가 후 컬럼 수: 30


In [8]:
# v2 후보 FE 1개씩 추가해서 baseline 대비 성능 비교

y_v2 = df_all_v2["is_churned"]

# baseline은 동일 (Set D)
X_base_v2 = df_all_v2[base_num + core_feats]

base_f1_v2, base_th_v2, base_auc_v2 = evaluate_rf_best_f1(X_base_v2, y_v2)
print("[Baseline Set D (v2)] F1 = {:.4f} @ th={:.2f}, AUC = {:.4f}".format(base_f1_v2, base_th_v2, base_auc_v2))

print("\n=== v2 후보 FE 1개씩 추가 (Set H_v2_x) ===")
for cand in CANDIDATE_FE_V2_COLS:
    if cand not in df_all_v2.columns:
        continue
    X_cand_v2 = df_all_v2[base_num + core_feats + [cand]]
    f1_c, th_c, auc_c = evaluate_rf_best_f1(X_cand_v2, y_v2)
    print(
        f"[+ {cand:28s}] F1 = {f1_c:.4f} (Δ {f1_c - base_f1_v2:+.4f}) @ th={th_c:.2f}, AUC = {auc_c:.4f} (Δ {auc_c - base_auc_v2:+.4f})"
    )


[Baseline Set D (v2)] F1 = 0.4127 @ th=0.10, AUC = 0.5289

=== v2 후보 FE 1개씩 추가 (Set H_v2_x) ===
[+ zero_offline_high_skip      ] F1 = 0.4115 (Δ -0.0012) @ th=0.10, AUC = 0.5343 (Δ +0.0054)
[+ very_low_engagement         ] F1 = 0.4112 (Δ -0.0015) @ th=0.15, AUC = 0.5365 (Δ +0.0076)
[+ ads_heavy_free              ] F1 = 0.4093 (Δ -0.0034) @ th=0.10, AUC = 0.5416 (Δ +0.0126)
[+ skip_to_engagement          ] F1 = 0.4111 (Δ -0.0016) @ th=0.10, AUC = 0.5415 (Δ +0.0126)
[+ ads_intensity_per_minute    ] F1 = 0.4117 (Δ -0.0010) @ th=0.10, AUC = 0.5322 (Δ +0.0033)
[+ engagement_x_skip           ] F1 = 0.4101 (Δ -0.0026) @ th=0.10, AUC = 0.5306 (Δ +0.0017)
[+ songs_per_min_x_ads_pressure] F1 = 0.4140 (Δ +0.0013) @ th=0.10, AUC = 0.5341 (Δ +0.0052)


---

## 최종 정리

위 v2 후보 실험까지 마친 후:

- **ΔF1이 +0.005 이상**으로 꾸준히 개선되는 FE가 있으면 → **채택 후보로 제안**
- 대부분 0 근처이거나 마이너스라면 → **현 데이터 기준 FE 탐색 종료**

이 노트북에서 확인한 내용:
1. 첫 후보(v1): 세그먼트 플래그, ratio, 비선형 → 성능 이득 거의 없음
2. 추가 후보(v2): 극단 플래그, 중첩 ratio, 교호작용 → 결과 확인 필요

**결론**: 두 라운드 실험 후에도 유의미한 개선이 없다면,  
→ **"현 데이터(유저당 1행 스냅샷)로는 FE를 더 추가해도 성능 한계에 도달했다"**고 정리하고,  
→ 파이프라인/모델 설계 단계로 넘어가는 것을 권장합니다.
